# River mask cleaner

The following code takes DSWE-derived watermasks from a specified local path. It also finds the associated "main_channel" vector produced from the Calculate_CFI notebook. Then, it overlays the main channel vector with the channel mask and selects the wetted portions of the mask intersecting the main channel, thus eliminating detected surface water not connected to the river. The processed mask is placed in a "Cleaned" folder. If no "Cleaned" folder exists, a new folder with that name is generated. 

Author: James (Huck) Rees; PhD Student, UCSB Geography

Date: January 16, 2025

## Import packages

In [1]:
import numpy as np
import rasterio
from scipy.ndimage import label as ndimage_label
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
from shapely.geometry import Polygon, MultiPoint
import os
from rasterio.transform import from_origin
import glob
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.plot import show
from shapely.ops import transform as shapely_transform
from pyproj import CRS, Transformer
from scipy.ndimage import label
from rasterio.enums import Resampling as RioResampling
from rasterio.io import MemoryFile

## Initialize functions

In [2]:
def import_raster(file_path):
    """
    Imports a raster file from a specified local path.

    Parameters:
    file_path (str): The path to the raster file.

    Returns:
    dataset: A rasterio dataset object.
    """
    try:
        dataset = rasterio.open(file_path)
        return dataset
    except rasterio.errors.RasterioIOError as e:
        print(f"Error opening raster file: {e}")
        return None

def classify_wet_components_array(array, min_pixels=200):
    """
    Classify connected components in a 2-D water mask array and eliminate small components.

    Parameters:
    array: A 2-D numpy array with pixel values 0 (dry) and 1 (wet).
    min_pixels (int): Minimum number of pixels for a component to be retained. Default is 200.

    Returns:
    classified_raster: A 2-D numpy array with unique integer values for each component.
    """
    structure = np.ones((3, 3), dtype=int)  # 8-connectivity
    labeled_array, num_features = ndimage_label(array, structure=structure)

    component_sizes = np.bincount(labeled_array.ravel())
    for component_label, size in enumerate(component_sizes):
        if size < min_pixels:
            labeled_array[labeled_array == component_label] = 0

    labeled_array, num_features = ndimage_label(labeled_array > 0, structure=structure)
    return labeled_array

def import_and_reproject_shapefile(shapefile_path, target_crs, default_crs='EPSG:4326'):
    """
    Import a shapefile and reproject it to the target CRS.
    
    Parameters:
    shapefile_path (str): Path to the shapefile
    target_crs: The target CRS (from raster) to reproject the shapefile to.
    default_crs (str): A default CRS to assign if the shapefile lacks a CRS (e.g., 'EPSG:4326' for WGS 84).
    
    Returns:
    shapefile (GeoDataFrame): Reprojected GeoDataFrame, or None if file is missing
    """
    # Check if the shapefile exists
    if not os.path.exists(shapefile_path):
        print(f"Shapefile not found: {shapefile_path}. Skipping this reach/year.")
        return None
    
    # Load shapefile
    shapefile = gpd.read_file(shapefile_path)
    
    # Check if the shapefile has a CRS set
    if shapefile.crs is None:
        print(f"The shapefile at {shapefile_path} does not have a CRS defined. Assigning default CRS: {default_crs}")
        shapefile.set_crs(default_crs, inplace=True)
    
    # Reproject shapefile if CRS does not match the target CRS
    if shapefile.crs != target_crs:
        shapefile = shapefile.to_crs(target_crs)
    
    return shapefile

def raster_to_gdf(labeled_array, transform):
    """
    Convert a labeled raster array to a GeoDataFrame with polygons for each pixel.

    Parameters:
    labeled_array: A 2-D numpy array with labeled components.
    transform: Affine transform for converting array coordinates to geographic coordinates.

    Returns:
    gdf: A GeoDataFrame with polygons for each pixel.
    """
    rows, cols = labeled_array.shape
    polygons = []
    labels = []

    for row in range(rows):
        for col in range(cols):
            value = labeled_array[row, col]
            if value != 0:  # Skip background pixels
                # Create a polygon for each pixel
                minx, miny = transform * (col, row)
                maxx, maxy = transform * (col + 1, row + 1)
                polygons.append(box(minx, miny, maxx, maxy))
                labels.append(value)

    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame({'geometry': polygons, 'label': labels})

    return gdf

def find_intersecting_pixels(raster_gdf, line_gdf):
    """
    Find unique pixel values that a line intersects with.

    Parameters:
    raster_gdf: GeoDataFrame with raster pixels as polygons.
    line_gdf: GeoDataFrame with polyline geometry.

    Returns:
    unique_labels: A set of unique pixel values that the line intersects.
    """
    # Perform a spatial join to find intersecting pixels
    intersections = gpd.sjoin(raster_gdf, line_gdf, how='inner', predicate='intersects')
    unique_labels = set(intersections['label'])
    
    return unique_labels

def filter_and_reclassify_raster(classified_raster, unique_labels, min_pixels=200):
    """
    Filter the classified raster to include only specified components and reclassify pixels.
    Eliminate components smaller than min_pixels.

    Parameters:
    classified_raster: A 2-D numpy array with labeled components.
    unique_labels: A set of unique pixel values to retain as wet.
    min_pixels (int): Minimum number of pixels for a component to be retained. Default is 200.

    Returns:
    final_raster: A 2-D numpy array with reclassified pixels.
    """
    # Create a new array for the final raster
    final_raster = np.zeros_like(classified_raster, dtype=np.int32)

    # Set pixels that are in unique_labels to 1 (wet)
    for unique_label in unique_labels:
        final_raster[classified_raster == unique_label] = 1

    # Label the connected components in the final raster
    structure = np.ones((3, 3), dtype=int)  # 8-connectivity
    labeled_array, num_features = ndimage_label(final_raster, structure=structure)

    # Eliminate components smaller than min_pixels
    component_sizes = np.bincount(labeled_array.ravel())
    for component_label, size in enumerate(component_sizes):
        if size < min_pixels:
            final_raster[labeled_array == component_label] = 0

    return final_raster

def export_raster(raster, transform, crs, output_path):
    """
    Export a raster array to a GeoTIFF file.

    Parameters:
    raster: A 2-D numpy array to export.
    transform: Affine transform for the raster.
    crs: Coordinate reference system of the raster.
    output_path (str): The path to save the GeoTIFF file.
    """
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=raster.shape[0],
        width=raster.shape[1],
        count=1,
        dtype=raster.dtype,
        crs=crs,
        transform=transform,
    ) as dst:
        dst.write(raster, 1)

def fill_small_dry_holes(array, max_size=10):
    mask_zeros = array == 0
    labeled, num_features = label(mask_zeros)
    for region in range(1, num_features + 1):
        if np.sum(labeled == region) <= max_size:
            array[labeled == region] = 1
    return array

def clip_raster_to_polygon(raster, polygon_gdf):
    """
    Clips a rasterio dataset to the extent of the polygon_gdf.

    Parameters:
    raster (rasterio.DatasetReader): The open raster dataset.
    polygon_gdf (GeoDataFrame): The polygon to clip the raster to.

    Returns:
    clipped_raster (numpy.ndarray): The clipped 2D array.
    clipped_transform (Affine): The updated affine transform.
    """
    polygon_gdf = polygon_gdf.to_crs(raster.crs)
    geom = [feature["geometry"] for feature in polygon_gdf.__geo_interface__["features"]]

    clipped_image, clipped_transform = mask(raster, geom, crop=True)
    return clipped_image[0], clipped_transform
    
def get_reach_gdf(base_shapefile_path, river_name, reach_number):
    """
    Returns a GeoDataFrame for a specific reach number using the 'ds_order' field.
    """
    reach_path = os.path.join(base_shapefile_path, "Reaches", river_name, f"{river_name}.shp")
    all_reaches_gdf = gpd.read_file(reach_path)

    return all_reaches_gdf[all_reaches_gdf['ds_order'] == reach_number]

def main(river_name, reach_range, year_range, base_raster_path, base_shapefile_path, min_pixels=200):
    river_folder = os.path.join(base_raster_path, river_name)

    if isinstance(reach_range, int):
        reach_folders = [f"reach_{reach_range}"]
    elif isinstance(reach_range, tuple):
        reach_folders = [f"reach_{i}" for i in range(reach_range[0], reach_range[1] + 1)]
    elif reach_range == "All":
        reach_folders = [d for d in os.listdir(river_folder) if d.startswith("reach_")]
    else:
        raise ValueError("Invalid reach_range input")

    for reach_folder in reach_folders:
        reach_number = int(reach_folder.split('_')[1])
        reach_path = os.path.join(river_folder, reach_folder, "Processed")

        if isinstance(year_range, int):
            years = [year_range]
        elif isinstance(year_range, tuple):
            years = range(year_range[0], year_range[1] + 1)
        elif year_range == "All":
            years = None
        else:
            raise ValueError("Invalid year_range input")

        raster_files = glob.glob(os.path.join(reach_path, "*.tif"))

        for raster_path in raster_files:
            raster_filename = os.path.basename(raster_path)
            try:
                year = int(raster_filename.split('_')[-4])
                dswe_level = raster_filename.split('_')[-1][0]
            except ValueError:
                continue

            if years is not None and year not in years:
                continue

            shapefile_path = os.path.join(base_shapefile_path, "Channels", river_name, f"reach_{reach_number}", str(year), "main_channel.shp")

            raster = import_raster(raster_path)
            reach_bnd = get_reach_gdf(base_shapefile_path, river_name, reach_number)
            clipped_raster, clipped_transform = clip_raster_to_polygon(raster, reach_bnd)
            classified_raster = classify_wet_components_array(clipped_raster, min_pixels=min_pixels)

            main_channel = import_and_reproject_shapefile(shapefile_path, raster.crs)
            if main_channel is None:
                continue

            raster_gdf = raster_to_gdf(classified_raster, clipped_transform)
            unique_labels = find_intersecting_pixels(raster_gdf, main_channel)
            filtered_raster = filter_and_reclassify_raster(classified_raster, unique_labels, min_pixels=min_pixels)
            final_raster = fill_small_dry_holes(filtered_raster, max_size=10)

            output_folder = os.path.join(base_raster_path, river_name, f"reach_{reach_number}", "Cleaned")
            os.makedirs(output_folder, exist_ok=True)

            final_raster_filename = f"{river_name}_reach_{reach_number}_{year}_DSWE_level_{dswe_level}_cleaned.tif"
            final_raster_path = os.path.join(output_folder, final_raster_filename)

            export_raster(final_raster, clipped_transform, raster.crs, final_raster_path)
            
def process(csv_file):
    """
    Processes multiple rivers based on input from a CSV file.
    
    Parameters:
    csv_file (str): Path to the CSV file with river information.

    The CSV is expected to have the following columns:
    - river_name: The name of the river.
    - working_directory: Path to the working directory.
    - reach_range: A tuple or integer specifying the range of reaches, or 'All' for all.
    - year_range: A tuple or integer specifying the range of years, or 'All' for all.
    - mask_output_path: Path to the raster mask file.
    - channel_files_output: Path to the vector shapefile.
    - min_pixels: Minimum number of pixels to keep a water component.
    """
    try:
        df = pd.read_csv(csv_file)
        
        # Ensure all required columns are present
        required_columns = ['river_name', 'working_directory', 'reach_range', 'year_range', 'min_pixels']
        for col in required_columns:
            if col not in df.columns:
                raise ValueError(f"Missing required column: {col}")
        
        # Iterate over each row and call the main function
        for index, row in df.iterrows():
            river_name = row['river_name']
            working_directory = row['working_directory']
            reach_range = row['reach_range']
            year_range = row['year_range']
            min_pixels = row['min_pixels']
            base_raster_path = os.path.join(working_directory, "RiverMapping", "RiverMasks")
            base_shapefile_path = os.path.join(working_directory, "RiverMapping")
            
            # Parse year_range and reach_range: Handle "All", single integer, and tuples
            # For reach_range
            if isinstance(reach_range, str) and reach_range == 'All':
                reach_range = 'All'  # Process all reaches
            elif isinstance(reach_range, str):
                # Attempt to parse reach_range as a tuple or integer
                try:
                    reach_range = eval(reach_range)
                except Exception:
                    raise ValueError(f"Invalid reach_range input: {reach_range}")
            
            # For year_range
            if isinstance(year_range, str) and year_range == 'All':
                year_range = 'All'  # Process all years
            elif isinstance(year_range, str):
                # Attempt to parse year_range as a tuple or integer
                try:
                    year_range = eval(year_range)
                except Exception:
                    raise ValueError(f"Invalid year_range input: {year_range}")

            # Print status for user tracking
            print(f"Processing river: {river_name}, reach_range: {reach_range}, year_range: {year_range}")
            
            # Call the main function for each river
            main(river_name, reach_range, year_range, base_raster_path, base_shapefile_path, min_pixels)
        
        print("Processing complete for all rivers.")
    
    except Exception as e:
        print(f"Error during processing: {e}")

In [3]:
csv_path = r"C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\Beni_river_datasheet.csv"
process(csv_path)

Processing river: Beni, reach_range: All, year_range: All
The shapefile at C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\Channels\Beni\reach_1\1984\main_channel.shp does not have a CRS defined. Assigning default CRS: EPSG:4326


C:\Users\huckr\AppData\Local\Temp\ipykernel_4012\3856295887.py:113: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  intersections = gpd.sjoin(raster_gdf, line_gdf, how='inner', predicate='intersects')


The shapefile at C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\Channels\Beni\reach_1\1986\main_channel.shp does not have a CRS defined. Assigning default CRS: EPSG:4326


C:\Users\huckr\AppData\Local\Temp\ipykernel_4012\3856295887.py:113: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  intersections = gpd.sjoin(raster_gdf, line_gdf, how='inner', predicate='intersects')


The shapefile at C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\Channels\Beni\reach_1\1987\main_channel.shp does not have a CRS defined. Assigning default CRS: EPSG:4326


...

The shapefile at C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\Channels\Beni\reach_9\2023\main_channel.shp does not have a CRS defined. Assigning default CRS: EPSG:4326


C:\Users\huckr\AppData\Local\Temp\ipykernel_4012\3856295887.py:113: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  intersections = gpd.sjoin(raster_gdf, line_gdf, how='inner', predicate='intersects')


The shapefile at C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\Channels\Beni\reach_9\2024\main_channel.shp does not have a CRS defined. Assigning default CRS: EPSG:4326


C:\Users\huckr\AppData\Local\Temp\ipykernel_4012\3856295887.py:113: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  intersections = gpd.sjoin(raster_gdf, line_gdf, how='inner', predicate='intersects')


Processing complete for all rivers.
